In [61]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import os, sys

PATH = "../Data/Images/"

img1 = cv.imread(f'{PATH}G1.JPG')
imgBis = cv.imread(f'{PATH}G14.JPG')
gray = cv.cvtColor(img1, cv.COLOR_BGR2GRAY)
# gray = cv.GaussianBlur(gray, (141,141),0)
grayBis = cv.cvtColor(imgBis, cv.COLOR_BGR2GRAY)

In [2]:
def show(img):
    cv.namedWindow('raw', cv.WINDOW_KEEPRATIO)
    cv.imshow('raw', img)
    cv.resizeWindow('raw', 1000, 1000)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [3]:
def findKeyLocationPosition(image_2darray, currKp_2darray, xMin, xMax, yMin, yMax):
    currKp_2darray[xMin:xMax, yMin:yMax] = 1
    return currKp_2darray

def findKeyLocationsColor(image_2darray, currKp_2darray, color_array):
    colorMask_2darray = np.mean(np.absolute(image_2darray - color_array), axis=2)< 20
    pointsDefect_2darray = np.asarray(np.where(colorMask_2darray == True)).T
    currKp_2darray = np.logical_and(currKp_2darray, colorMask_2darray)
    
    return currKp_2darray

def plotKeypoints(image, keypoints_2darray):
    iterable_new = []
    keypoints = np.asarray(np.where(keypoints_2darray == True)).T
    for idx, val in enumerate(keypoints):

        iterable_line = [np.uint32(keypoints[idx][1]).item(), np.uint32(keypoints[idx][0]).item()]
        iterable_new.append(iterable_line)
        
    area1 = cv.KeyPoint_convert(iterable_new)
    imagePlot = cv.drawKeypoints(image, area1, None, color=(255,0,0), flags=0)
    
    show(imagePlot)
    return

In [13]:
corners = cv.goodFeaturesToTrack(img1,25,0.01,10)

corners = np.int0(corners)
for i in corners:
    x,y = i.ravel()
    cv.circle(img1,(x,y),3,255,-1)

# plt.figure(figsize = (20,20))
# plt.imshow(img1),plt.show()

In [7]:
# Initiate FAST object with default values
fast = cv.FastFeatureDetector_create()

# find and draw the keypoints
kp = fast.detect(img1,None)
img2 = cv.drawKeypoints(img1, kp, None, color=(255,0,0))

show(img2)

In [4]:
# Initiate ORB detector
orb = cv.ORB_create(nfeatures = 30, scaleFactor = 1.2, edgeThreshold=400, scoreType=cv.ORB_HARRIS_SCORE, patchSize=400) #scoreType=cv.ORB_FAST_SCORE
# find the keypoints with ORB
kp = orb.detect(gray, positionKp_Conv)
# compute the descriptors with ORB
kp, des = orb.compute(gray, kp)
# draw only keypoints location,not size and orientation
img2 = cv.drawKeypoints(gray, kp, None, color=(0,255,0), flags=0)

show(img2)

NameError: name 'positionKp_Conv' is not defined

In [ ]:
th3 = cv.adaptiveThreshold(gray,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv.THRESH_BINARY,101,2)
show(th3)

sharp_img = cv.createBackgroundSubtractorMOG2().apply(th3)
show(sharp_img)

In [22]:
show(gray)

In [27]:
initialKp = np.zeros(img1.shape, dtype=np.uint8)[:,:,0]
positionKp = findKeyLocationPosition(img1, initialKp, 520, 1036, 633, 1051)
positionKp_Conv = positionKp*255

# cutImg1 = gray[520:1036, 633:1051].copy()
# cutImg2 = grayBis[520:1036, 633:1051].copy()

cutImg1 = gray
cutImg2 = grayBis

# colourKp = findKeyLocationsColor(img1, initialKp, color_array=[46,114,168])

# print(colourKp, initialKp)
# plotKeypoints(img1, positionKp)


In [56]:
import numpy as np
import cv2 as cv

cv.namedWindow('ORB', cv.WINDOW_NORMAL)

def f(x):
    return

# Initiate ORB detector
cv.createTrackbar('Edge Threshold', 'ORB', 100, 1000, f)
cv.createTrackbar('Patch Size', 'ORB', 100, 200, f)
cv.createTrackbar('N Levels', 'ORB', 10, 30, f)
cv.createTrackbar('Fast Threshold', 'ORB', 15, 50, f)
cv.createTrackbar('Scale Factor', 'ORB', 12, 25, f)
cv.createTrackbar('WTA K', 'ORB', 2, 4, f)
cv.createTrackbar('First Level', 'ORB', 0, 20, f)
cv.createTrackbar('N Features', 'ORB', 250, 500, f)

while True:
    edge_threshold = cv.getTrackbarPos('Edge Threshold', 'ORB')
    patch_size = cv.getTrackbarPos('Patch Size', 'ORB')
    n_levels = cv.getTrackbarPos('N Levels', 'ORB')
    fast_threshold = cv.getTrackbarPos('Fast Threshold', 'ORB')
    scale_factor = cv.getTrackbarPos('Scale Factor', 'ORB') / 10
    wta_k = cv.getTrackbarPos('WTA K', 'ORB')
    first_level = cv.getTrackbarPos('First Level', 'ORB')
    n_features = cv.getTrackbarPos('N Features', 'ORB')

    if wta_k < 2:
        wta_k = 2

    if patch_size < 2:
        patch_size = 2

    if n_levels < 1:
        n_levels = 1

    if scale_factor < 1:
        scale_factor = 1

    orb = cv.ORB_create(edgeThreshold=edge_threshold, patchSize=patch_size, nlevels=n_levels, fastThreshold=fast_threshold, scaleFactor=scale_factor, WTA_K=wta_k,scoreType=cv.ORB_HARRIS_SCORE, firstLevel=first_level, nfeatures=n_features)
    
    #Orb parameters patch1 100, 100, 6, 15, 12, 2, 0, 50
    #Orb parameters fullImg 100, 100, 10, 15, 12, 2, 0, 250
    
    # find the keypoints with ORB
    kp1 = orb.detect(cutImg1,None)
    kp2 = orb.detect(cutImg2,None)
    # compute the descriptors with ORB
    kp1, des1 = orb.compute(cutImg1, kp1)
    kp2, des2 = orb.compute(cutImg2, kp2)

    # draw only keypoints location,not size and orientation
    img1B = cv.drawKeypoints(cutImg1, kp1, None, color=(0,255,0), flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS).copy()
    img2B = cv.drawKeypoints(cutImg2, kp2, None, color=(0,255,0), flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS).copy()
    
    finalImg = np.concatenate((img1B, img2B), axis=1)
    
    cv.imshow('ORB', finalImg)
    
    if cv.waitKey(10) & 0xFF == 27:
        break

cv.destroyAllWindows()

In [57]:
# BFMatcher with default params
bf = cv.BFMatcher()
matches = bf.knnMatch(des1,des2,k=2)
# Apply ratio test
good = []
for m,n in matches:
    if m.distance < 0.75*n.distance:
        good.append([m])
# cv.drawMatchesKnn expects list of lists as matches.
img3 = cv.drawMatchesKnn(cutImg1,kp1,cutImg2,kp2,good,None,flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
show(img3)


#Flann

# FLANN_INDEX_LSH = 6
# index_params= dict(algorithm = FLANN_INDEX_LSH,
#                    table_number = 6, # 12
#                    key_size = 12,     # 20
#                    multi_probe_level = 1) #2

# search_params = dict(checks=50)   # or pass empty dictionary
# flann = cv.FlannBasedMatcher(index_params,search_params)
# matches = flann.knnMatch(des1,des2,k=2)

# # Need to draw only good matches, so create a mask
# matchesMask = [[0,0] for i in range(len(matches))]
# # ratio test as per Lowe's paper
# for i,(m,n) in enumerate(matches):
#     if m.distance < 0.7*n.distance:
#         matchesMask[i]=[1,0]
# draw_params = dict(matchColor = (0,255,0),
#                    singlePointColor = (255,0,0),
#                    matchesMask = matchesMask,
#                    flags = cv.DrawMatchesFlags_DEFAULT)

# img3 = cv.drawMatchesKnn(img1,kp1,img2,kp2,matches,None,**draw_params)

# show(img3)





In [59]:
MIN_MATCH_COUNT = 10

good = np.squeeze(good)

if len(good)>MIN_MATCH_COUNT:
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
    M, mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC,5.0)
    matchesMask = mask.ravel().tolist()
    h,w = cutImg1.shape
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    dst = cv.perspectiveTransform(pts,M)
    img2 = cutImg2.copy()
    img2 = cv.polylines(img2,[np.int32(dst)],True,255,3, cv.LINE_AA)
else:
    print( "Not enough matches are found - {}/{}".format(len(good), MIN_MATCH_COUNT) )
    matchesMask = None
    
finalImg = np.concatenate((cutImg1, cutImg2, img2), axis=1)

show(finalImg)

In [60]:
if len(good)>MIN_MATCH_COUNT:
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
    M, mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC,5.0)

    h,w = gray.shape
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    img2 = grayBis.copy()
    img2 = cv.warpPerspective(img2, M, img2.T.shape, flags=cv.INTER_LINEAR)
    
else:
    print( "Not enough matches are found - {}/{}".format(len(good), MIN_MATCH_COUNT) )
    matchesMask = None
    
finalImg = np.concatenate((gray, grayBis, img2), axis=1)

show(finalImg)